In [1]:
import sys
sys.path.append("../") # go to parent dir

%load_ext autoreload
%autoreload 2

In [2]:
from evrp.utils.utilities import parse_csv_tables, calculate_distance_matrix, generate_index_mapping, create_plotting_edges, create_flat_distance_matrix, create_coordinate_dict, create_flat_optimal_edges, create_optimal_edges, merge_variable_results
from evrp.utils.graph import draw_plain_graph, draw_graph
from evrp.utils.plot import plot_interactive_graph
from evrp.milp.tsp import TSP
from evrp.milp.vrptw import VRPTW
from evrp.milp.evrptw import EVRPTW

import pandas as pd
import numpy as np
from scipy.spatial import distance

import pprint

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.colors as pltc

import itertools

import networkx as nx

from pyomo.environ import *

pd.options.plotting.backend = "plotly"
pp = pprint.PrettyPrinter()

In [3]:
instance = 'r105C5_e'
fpath = 'config/test_instances/' + instance + '.csv'

In [418]:
m = EVRPTW()
m.solve(fpath)

2021-01-13 09:08:20 INFO     Building abstract model
2021-01-13 09:08:20 INFO     Defining parameters and sets
2021-01-13 09:08:20 INFO     Defining variables
2021-01-13 09:08:20 INFO     Defining constraints
2021-01-13 09:08:20 INFO     Defining objective
2021-01-13 09:08:20 INFO     Importing EVRPTW MILP instance: r105C5_e
2021-01-13 09:08:20 INFO     Reading CSV
2021-01-13 09:08:20 INFO     Creating graph
2021-01-13 09:08:20 INFO     Creating timeseries data
2021-01-13 09:08:20 INFO     Creating duplicates and extended graph
2021-01-13 09:08:20 INFO     Calculating distance matrix
2021-01-13 09:08:20 INFO     Creating parameters
2021-01-13 09:08:20 INFO     Creating instance
2021-01-13 09:08:20 INFO     Solving instance...


Using license file /Users/ramiariss/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-28
Read LP format model from file /var/folders/vv/1p3h17yx43n9m7k86th68_ym0000gn/T/tmp_bgzbj1q.pyomo.lp
Reading time = 0.01 seconds
x1484: 202 rows, 1484 columns, 654 nonzeros
No parameters matching 'MaxTime' found
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 202 rows, 1484 columns and 654 nonzeros
Model fingerprint: 0xf3d13bd1
Model has 2808 quadratic constraints
Variable types: 721 continuous, 763 integer (763 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+02]
  Objective range  [4e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
  QRHS range       [2e+00, 2e+02]
Presolve removed 88 rows and 26 columns
Presolve time: 0.01s
Presolved: 1497 rows, 3528 column

2021-01-13 09:08:20 WARNING  Loading a SolverResults object with a warning status into model=unknown;
    message from solver=Model was proven to be infeasible.
2021-01-13 09:08:20 INFO     Done


    model=unknown;
        message from solver=Model was proven to be infeasible.


In [404]:
print(m.results)


Problem: 
- Name: x1484
  Lower bound: 151.14879946121792
  Upper bound: 151.14879946121792
  Number of objectives: 1
  Number of constraints: 3010
  Number of variables: 1484
  Number of binary variables: 763
  Number of integer variables: 763
  Number of continuous variables: 721
  Number of nonzeros: 654
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.030688047409057617
  Error rc: 0
  Time: 0.17081022262573242
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [405]:
var_list = ['xkappa', 'xp']
x = merge_variable_results(m, var_list)

In [406]:
x[x['state']>1e-4].sort_values(['xp'], ascending=False)

,node,t,state,xp
0,S0_0,0,1.0,2.040816
463,S13_0,3,1.0,2.040816
455,S1_0,225,1.0,2.040816
456,S1_0,226,1.0,2.040816
457,S1_0,227,1.0,2.040816
...,...,...,...,...
231,S1_0,1,1.0,2.040816
232,S1_0,2,1.0,2.040816
233,S1_0,3,1.0,2.040816
234,S1_0,4,1.0,2.040816


In [407]:
var_list = ['xgamma', 'xw', 'xq', 'xa']
x = merge_variable_results(m, var_list)

In [408]:
x[x['state']>1e-4].sort_values(['xw'])

,from,to,state,xw,xq,xa
49,D0,C75,1.0,49.000000,200.0,72.215112
38,C75,C91,1.0,93.928498,182.0,37.286614
69,C91,C95,1.0,130.000000,39.0,26.106274
57,D0,C78,1.0,178.000000,19.0,31.324555
47,C78,C28,1.0,213.000000,16.0,6.324555
5,C28,D1,1.0,230.000000,0.0,0.000000
8,C95,D1,1.0,230.000000,0.0,0.000000


In [409]:
test = x[x['state']>1e-4].sort_values(['xw'])
route = zip(test['from'], test['to'])
sum([m.data['d'].loc[fr, to] for fr, to in route])

151.14879946121792

In [410]:
m.data['d']

,D0,D1,S0_0,S1_0,S13_0,C91,C28,C75,C78,C95
D0,0.000000,0.000000,0.000000,29.068884,19.104973,25.612497,6.324555,27.784888,31.064449,14.866069
D1,0.000000,0.000000,0.000000,29.068884,19.104973,25.612497,6.324555,27.784888,31.064449,14.866069
S0_0,0.000000,0.000000,0.000000,29.068884,19.104973,25.612497,6.324555,27.784888,31.064449,14.866069
S1_0,29.068884,29.068884,29.068884,0.000000,45.541190,52.201533,23.000000,30.016662,15.297059,41.109610
S13_0,19.104973,19.104973,19.104973,45.541190,0.000000,6.708204,25.000000,30.083218,50.000000,4.472136
C91,25.612497,25.612497,25.612497,52.201533,6.708204,0.000000,31.622777,34.928498,56.612719,11.180340
C28,6.324555,6.324555,6.324555,23.000000,25.000000,31.622777,0.000000,27.202941,25.000000,20.615528
C75,27.784888,27.784888,27.784888,30.016662,30.083218,34.928498,27.202941,0.000000,42.720019,27.294688
C78,31.064449,31.064449,31.064449,15.297059,50.000000,56.612719,25.000000,42.720019,0.000000,45.607017
C95,14.866069,14.866069,14.866069,41.109610,4.472136,11.180340,20.615528,27.294688,45.607017,0.000000


In [411]:
25.612497+11.180340+14.866069

51.658906

In [412]:
e, e_flat = create_optimal_edges(m, x=x, graph='V_')

In [413]:
plot_interactive_graph(m.data['V'], e=e_flat, obj=m.instance.obj(), instance_name=m.instance_name)

In [313]:
v = 'constraint_energy_customer'
x = getattr(m.instance, v)
keys, values = zip(*x.get_values().items())

AttributeError: 'IndexedConstraint' object has no attribute 'get_values'

In [104]:
m.instance.constraint_energy_customer.pprint()

constraint_energy_customer : Size=49, Index=constraint_energy_customer_index, Active=True
    Key              : Lower : Body                                                                                          : Upper : Active
      ('C28', 'C75') :  -Inf :       xa[C75] - (xa[C28] - 27.202941017470888*xgamma[C28,C75] + 5000.0*(1 - xgamma[C28,C75])) :   0.0 :   True
      ('C28', 'C78') :  -Inf :                     xa[C78] - (xa[C28] - 25.0*xgamma[C28,C78] + 5000.0*(1 - xgamma[C28,C78])) :   0.0 :   True
      ('C28', 'C91') :  -Inf :       xa[C91] - (xa[C28] - 31.622776601683793*xgamma[C28,C91] + 5000.0*(1 - xgamma[C28,C91])) :   0.0 :   True
      ('C28', 'C95') :  -Inf :       xa[C95] - (xa[C28] - 20.615528128088304*xgamma[C28,C95] + 5000.0*(1 - xgamma[C28,C95])) :   0.0 :   True
       ('C28', 'D1') :  -Inf :           xa[D1] - (xa[C28] - 6.324555320336759*xgamma[C28,D1] + 5000.0*(1 - xgamma[C28,D1])) :   0.0 :   True
     ('C28', 'S0_0') :  -Inf :     xa[S0_0] - (xa[C28] - 6

In [105]:
m.instance.constraint_energy_start_end_soe.pprint()

AttributeError: 'ConcreteModel' object has no attribute 'constraint_energy_start_end_soe'

In [106]:
m.instance.xkappa.pprint()

xkappa : Size=690, Index=xkappa_index
    Key            : Lower : Value : Upper : Fixed : Stale : Domain
       ('S0_0', 0) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 1) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 2) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 3) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 4) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 5) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 6) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 7) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 8) :     0 :   0.0 :     1 : False : False : Boolean
       ('S0_0', 9) :     0 :   0.0 :     1 : False : False : Boolean
      ('S0_0', 10) :     0 :   0.0 :     1 : False : False : Boolean
      ('S0_0', 11) :     0 :   0.0 :     1 : False : False : Boolean
      ('S0_0', 12) :     0 :   0.0 :     1 : False : False : Boole

In [312]:
m.instance.xa['D0'].pprint()

{Member of xa} : Size=10, Index=V01_
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     D0 :     0 : 100.0 :  None : False : False : NonNegativeReals
